# Structural analysis of discontinuous concepts

The purpose of this analysis notebook is to make an in-depth analysis of structures. This makes the basis for a large portion of the corresponding section in the final thesis paper.

## Coordinated concepts

Coordinations are the only thing marked in GENIA while CRAFT also contains other types. Therefore, GENIA will be used primarily in this part. The variable `corpus_name` can be set to either `genia` or `craft` to run with.

In [1]:
import utils  # a python module in the same dir as the notebooks

In [2]:
corpus_name = 'acl'

First, let's load the corpus, then extract concepts and their constituent structures.

In [3]:
import os
os.chdir(utils.ROOT)  # get to the root directory of the project

from datautils import dataio, annotations as anno

# load the corpora
corpus = dataio.load_corpus(corpus_name)

# not all docs in genia get Constituent annotations; if so, leave them out
if corpus_name.lower() == 'genia':
    corpus = [doc for doc in corpus if doc.get_annotations(anno.Constituent)]

Loading ACL 2.0 corpus: 100%|██████████| 300/300 [00:00<00:00, 518.17it/s]


In [4]:
concept_consts = [(concept, doc.get_annotations_at(concept.span, anno.Constituent)[0])
                  for doc in corpus
                  for concept in doc.get_annotations(anno.DiscontinuousConcept)
                  if len(concept.get_spanned_tokens()) < 10]

# for craft, keep only the ones with a conjunction in them
temp_list = []
for concept, const in concept_consts:
    present_pos = {t.mapped_pos() for t in const.get_tokens()}
    if 'c' in present_pos:
        temp_list.append((concept, const))
concept_consts = temp_list

Each constituent can spit out a treebank structure. Let's do that and put it all in dataframes.

In [5]:
import re

def skipped_structure(const, allowed_tokens):
    """Returns  a treebank structure of a Constituent with a # symbol in front of
    all tokens that are in allowed_tokens"""
    # note: craft constituents often have labels like NP-SBJ; keep only the first part
    return '(' + const.label.split('-')[0] + ' ' + ' '.join(
        skipped_structure(c, allowed_tokens) if isinstance(c, anno.Constituent)
        else c.mapped_pos() if c not in allowed_tokens
        else '#' + c.mapped_pos()
        for c in const.constituents
    ) + ')'

def unlabeled_structure(const, allowed_tokens):
    """Returns a simplified treebank structure of a Constituent with no constituent labels
    and with a # symbol in front of all tokens that are in allowed_tokens"""
    return_string = '(' + ' '.join(
        unlabeled_structure(c, allowed_tokens) if isinstance(c, anno.Constituent)
        else c.mapped_pos() if c not in allowed_tokens
        else '#' + c.mapped_pos()
        for c in const.constituents
        if not contains_empty_token(c)
    ) + ')'
    # remove all cases of single token constituents, e.g. (n)
    return re.sub('\(+(#?.)\)+', r'\1', return_string)

def contains_empty_token(const):
    if not isinstance(const, anno.Constituent):
        return False
    for c in const.constituents:
        if isinstance(c, anno.Token) and c.mapped_pos() == 'Ø':
            return True
    return False

In [6]:
import pandas as pd
from collections import defaultdict

full_structures = defaultdict(list)
skip_structures = defaultdict(list)
collapsed_full_structures = defaultdict(list)
collapsed_skip_structures = defaultdict(list)

for concept, const in concept_consts:
    struct = const.structure()  
    full_structures[struct].append(const)
    
    skip_struct = skipped_structure(const, concept.get_tokens())
    skip_structures[skip_struct].append(const)
    
    collapsed_full = unlabeled_structure(const, {})
    collapsed_full_structures[collapsed_full].append(const)
    
    collapsed_skip = unlabeled_structure(const, concept.get_tokens())
    collapsed_skip_structures[collapsed_skip].append(const)

In [7]:
n_dcs = len(concept_consts)

dfs = {}

for name, structure_dict in [('full', full_structures),
                             ('skip', skip_structures),
                             ('col_full', collapsed_full_structures),
                             ('col_skip', collapsed_skip_structures)]:
    
    data_dict = {'struct': [], 'pos-seq': [], 'count': [], '%': [], 'example': []}
    
    for struct, sample in structure_dict.items():
        data_dict['struct'].append(struct)
        data_dict['pos-seq'].append(''.join(c for c in struct if c.islower() or c in ','))
        data_dict['count'].append(len(sample))
        data_dict['%'].append(round(len(sample) / n_dcs * 100, 2))
        data_dict['example'].append(sample[0].get_covered_text())
        
    dfs[name] = pd.DataFrame(data_dict)

In [8]:
dfs['col_skip'].sort_values('count', ascending=False).head(50)

,struct,pos-seq,count,%,example


In [9]:
# for latex table
import re
csv_string = dfs['full'].sort_values('count', ascending=False).head(15).to_csv(sep='&')
csv_string = re.sub(r'\n\d+&', r'  \\\\ \\addlinespace\n', csv_string)  # line breaks and row numbers
csv_string = re.sub('^&', r'', csv_string)  # first &
csv_string = re.sub('%', r'\\%', csv_string)  # \% for latex
csv_string = re.sub('Ø', r'\\O', csv_string)  # \O for latex
csv_string = re.sub('#([A-Za-z\-]+)', r'\\textbf{\1}', csv_string)  # bold typeface used words 
csv_string = re.sub('&', r'\t&\t', csv_string)  # more space on delimiters

In [ ]:
print(csv_string)

In [ ]:
collapsed_skip_structures['((#a c a) #n)']

We can capture more examples under fewer labels if we only use POS-tag sequences.

In [ ]:
CROSS_COUNTING = ['concept', 'super', 'skip']  # order: concept, super, skip

MOD = '[an]'
SUPER_SEQ_COLLAPSERS = {
    # re.compile('n*(n,?)+cnn+'),
    # re.compile('a*(a,?)+caa*n+')
    #re.compile('[an]c[an]n'),  # base case: shared head
    #re.compile('[an]ncn'),  # base case: shared modifier
    #re.compile('([an],)+[an],?c[an]n'),  # exp case: enumeration of modifiers + shared head
    #re.compile('[an]+(n,)+n,?cn'),  # exp case: enumeration of heads with shared modifier
    #re.compile('[an]c[an][an]+n'),  # exp case: multi-word head
    #re.compile('[an]+[an]ncn'),  # exp case: shared pre-modifiers
    re.compile(f'({MOD}n?,?)+c{MOD}*n'),  # all
}

CONCEPT_SEQ_COLLAPSERS = {
    re.compile('(a|n)+n')
}
SAMPLE_CATEGORIES = {'concept', 'super', 'skip', 'cross-count'}

def make_sequence(tokens, collapser=None):
    sequence = [t.mapped_pos() if not t.pos == '#' else '#'
                for t in tokens]
    sequence_str = ''.join(sequence)
    if not collapser:
        return sequence_str

    for pos_pattern in collapser:
        if pos_pattern.fullmatch(sequence_str):
            return pos_pattern.pattern
    return sequence_str  # sequence did not match a pattern

In [ ]:
disc_concepts = [dc for doc in corpus
                 for dc in doc.get_annotations(anno.DiscontinuousConcept)
                 if 'c' in {t.mapped_pos() for t in dc.get_spanned_tokens()}
                 and len(dc.get_tokens()) < 10]

samples_after_category = {}
for ct in SAMPLE_CATEGORIES:
    samples_after_category[ct] = defaultdict(list)

for dc in disc_concepts:
    concept_tokens = dc.get_tokens()
    all_tokens = dc.get_spanned_tokens()
    skip_tokens = [t if t in concept_tokens  # actual token
                   else anno.Token(dc.document, (-1, -1), '#')  # skipped token
                   for t in all_tokens]

    cross_count_type = []
    cross_count_example = []

    concept_sequence = make_sequence(concept_tokens, CONCEPT_SEQ_COLLAPSERS)
    concept_text = dc.get_covered_text()
    samples_after_category['concept'][concept_sequence].append(dc)
    if 'concept' in CROSS_COUNTING:
        cross_count_type.append(concept_sequence)
        cross_count_example.append(dc)

    super_sequence = make_sequence(all_tokens, SUPER_SEQ_COLLAPSERS)
    super_text = dc.get_spanned_text()
    samples_after_category['super'][super_sequence].append(dc)
    if 'super' in CROSS_COUNTING:
        cross_count_type.append(super_sequence)
        cross_count_example.append(dc)

    skip_sequence = make_sequence(skip_tokens)
    samples_after_category['skip'][skip_sequence].append(dc)
    if 'skip' in CROSS_COUNTING:
        cross_count_type.append(skip_sequence)
        cross_count_example.append(dc)

    samples_after_category['cross-count'][tuple(cross_count_type)].append(
        tuple(cross_count_example)
    )

In [ ]:
data_dict = {'type': [], 'count': [], '%': [], 'example': []}
for type_, items in samples_after_category['super'].items():
    data_dict['type'].append(type_)
    data_dict['count'].append(len(items))
    data_dict['%'].append(round(len(items) / len(disc_concepts) * 100, 2))
    data_dict['example'].append(items[0].get_spanned_text())
    
df = pd.DataFrame(data_dict)

In [ ]:
df.sort_values('count', ascending=False).head(50)

In [ ]:
for item in samples_after_category['super']['[an]+[an]ncn']:
    print(*(t.get_covered_text() for t in item.get_spanned_tokens()))

In [ ]:
# for exp case: multi-word head: for multiple tokens before/after the conjunction,
# do the tokens go with the head or the second modifier?
df = dfs['col_full']
subset = df[df['pos-seq'].str.contains('^[an]+[an]ncn$')]
subset.head(25)

What about CC's?'

In [ ]:
all_concepts = [c for doc in corpus for c in doc.get_annotations(anno.Concept)
               if not isinstance(c, anno.DiscontinuousConcept) and not len(c) < 2]


In [ ]:
from collections import Counter
dist_of_types = Counter(c.pos_sequence() for c in all_concepts)

In [ ]:
dist_of_types.most_common(10)

In [ ]:
simple_count = sum(v for t, v in dist_of_types.items() if re.match('[an]n', t))

In [ ]:
simple_count / sum(dist_of_types.values())